# <font color='red'>Backpropagation</font>


In this assignment, you will implement Backpropagation from scratch. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you write and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% weightage in the final marks for this course.

## <font color='red'>Loading data </font>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import math

with open('/content/drive/MyDrive/AppliedRootsAIML/Semester2/Backpropagation_from_Scratch_Assignment_Data.pkl','rb') as f:
    data = pickle.load(f)
print(data.shape)
X = data[:, :5]
y = data[:, -1]
print(X.shape, y.shape)

In [ ]:
print(X[0])

<font color='blue'><b>Check this video for better understanding of the computational graphs and back propagation</font>

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('i94OvYb6noo',width="1000",height="500")

# <font color='red'>Computational graph</font>

<img src='https://i.imgur.com/seSGbNS.png'>


*  **If you observe the graph, we are having input features [f1, f2, f3, f4, f5] and 9 weights [w1, w2, w3, w4, w5, w6,    w7, w8, w9]**.<br><br>
*  **The final output of this graph is a value L which is computed as (Y-Y')^2** 

## <font color='red'>Task 1: Implementing Forward propagation, Backpropagation and Gradient checking </font>

# <font color='red'>Task 1.1 </font>

## <font color='blue'>Forward propagation </font>


*  <b>
    Forward propagation</b>(Write your code in<font color='blue'> def forward_propagation()</b></font>)<br><br>
    For easy debugging, we will break the computational graph into 3 parts.

    <font color='green'><b>Part 1</b></font></b>
    <img src='https://i.imgur.com/0xUaxy6.png'><br><br>
    <font color='green'><b>Part 2</b></font></b><br>
    <img src='https://i.imgur.com/J29pAJL.png'><br><br>
    <font color='green'><b>Part 3</b></font></b>
    <img src='https://i.imgur.com/vMyCsd9.png'>

In [ ]:
def sigmoid(z):
    '''In this function, we will compute the sigmoid(z)'''
    # we can use this function in forward and backward propagation
    # write the code to compute the sigmoid value of z and return that value 
    return 1/(1+np.exp(-z))

In [ ]:
def grader_sigmoid(z):
  #if you have written the code correctly then the grader function will output true
  val=sigmoid(z)
  assert(val==0.8807970779778823)
  return True
grader_sigmoid(2)

In [ ]:
def forward_propagation(x, y, w):
        '''In this function, we will compute the forward propagation '''
        # X: input data point, note that in this assignment you are having 5-d data points
        # y: output varible
        # W: weight array, its of length 9, W[0] corresponds to w1 in graph, W[1] corresponds to w2 in graph,..., W[8] corresponds to w9 in graph.  
        # you have to return the following variables
        # exp= part1 (compute the forward propagation until exp and then store the values in exp)
        # tanh =part2(compute the forward propagation until tanh and then store the values in tanh)
        # sig = part3(compute the forward propagation until sigmoid and then store the values in sig)
        # we are computing one of the values for better understanding
        
        val_1= (w[0]*x[0]+w[1]*x[1]) * (w[0]*x[0]+w[1]*x[1]) + w[5]
        part_1 = np.exp(val_1)
        val_2= part_1 + w[6]
        part_2 = np.tanh(val_2)
        val_3 = (np.sin(w[2]*x[2]) * ((w[3] * x[3]) + (w[4] * x[4]))) + w[7]
        part_3= sigmoid(val_3)
        
        # after computing part1,part2 and part3 compute the value of y' from the main Computational graph using required equations
        y_pred = part_2 + (part_3 * w[8])        
        # write code to compute the value of L=(y-y')^2 and store it in variable loss
        L = (y - y_pred)**2
        # compute derivative of L  w.r.to y' and store it in dy_pred 
        dy_pred = 2 * (y - y_pred) * (-1)
        # Create a dictionary to store all the intermediate values i.e. dy_pred ,loss,exp,tanh,sigmoid
        # we will be using the dictionary to find values in backpropagation, you can add other keys in dictionary as well
        
        forward_dict={}
        forward_dict['exp']= part_1
        forward_dict['sigmoid'] = part_3
        forward_dict['tanh'] = part_2
        forward_dict['loss'] = L
        forward_dict['dy_pred'] = dy_pred
        
        return forward_dict

In [ ]:
def grader_forwardprop(data):
    dl = (data['dy_pred']==-1.9285278284819143)
    loss=(data['loss']==0.9298048963072919)
    part1=(data['exp']==1.1272967040973583)
    part2=(data['tanh']==0.8417934192562146)
    part3=(data['sigmoid']==0.5279179387419721)
    assert(dl and loss and part1 and part2 and part3)
    return True
w=np.ones(9)*0.1
d1=forward_propagation(X[0],y[0],w)
grader_forwardprop(d1)

# <font color='red'>Task 1.2 </font>

## <font color='blue'>Backward propagation</font>

In [ ]:
# def backward_propagation(x,y,w,forward_dict):
#     '''In this function, we will compute the backward propagation '''
#     # forward_dict: the outputs of the forward_propagation() function
#     # write code to compute the gradients of each weight [w1,w2,w3,...,w9]
#     # Hint: you can use dict type to store the required variables 
#     # dw1 = # in dw1 compute derivative of L w.r.to w1
#     # dw2 = # in dw2 compute derivative of L w.r.to w2
#     # dw3 = # in dw3 compute derivative of L w.r.to w3
#     # dw4 = # in dw4 compute derivative of L w.r.to w4
#     # dw5 = # in dw5 compute derivative of L w.r.to w5
#     # dw6 = # in dw6 compute derivative of L w.r.to w6
#     # dw7 = # in dw7 compute derivative of L w.r.to w7
#     # dw8 = # in dw8 compute derivative of L w.r.to w8
#     # dw9 = # in dw9 compute derivative of L w.r.to w9
#     dw9 = forward_dict['dy_pred'] * 1 * forward_dict['sigmoid']
#     dO3O4 = forward_dict['sigmoid'] * (1- forward_dict['sigmoid'])
#     dLO4 = forward_dict['dy_pred'] * 1 * 0.1 * dO3O4
#     dw8 = dLO4 * 1
#     dLO6 = forward_dict['dy_pred'] * 1 * (1 - (forward_dict['tanh']**2))
#     dw7 = 1 * dLO6
#     dLO12 = forward_dict['exp'] * 1 * dLO6
#     dw6 = 1 * dLO12
#     dLO16 = 1 * (-0.002546) * 1 * dLO4
#     dw5 = -0.14421743 * dLO16
#     dLO15 = np.round(1 * (-0.002546) * 1 * dLO4,6)
#     #dw4 = -0.66660821 * dLO15    
#     dw4 = np.round(-0.666608 * dLO15,6)    
#     O18 = (x[3] * w[3]) + (x[4] * w[4])
#     dLO17 = O18 * 1 * dLO4    
#     dLO14 = math.cos(x[2] * w[2]) * dLO17
#     #dLO14 = 0.999997 * dLO17
#     dw3 = x[2] * dLO14
#     #dw3 = -1.45900038 * dLO14
#     dLO8 = (1 * (-0.140792) * 1 * dLO12) + (1 * (-0.140792) * 1 * dLO12)
#     dw2 = -0.12001342 * dLO8
#     dLO7 = dLO8
#     dw1 = -1.2879095 * dLO7
#     backward_dict={}
#     #store the variables dw1,dw2 etc. in a dict as backward_dict['dw1']= dw1,backward_dict['dw2']= dw2...
#     backward_dict['dw1']= dw1
#     backward_dict['dw2']= dw2
#     backward_dict['dw3']= dw3
#     backward_dict['dw4']= dw4
#     backward_dict['dw5']= dw5
#     backward_dict['dw6']= dw6
#     backward_dict['dw7']= dw7
#     backward_dict['dw8']= dw8
#     backward_dict['dw9']= dw9
#     #print(dw1 + ","+ dw2+ ","+dw3+ ","+dw4+ ","+dw5+ ","+dw6+ ","+dw7+ ","+dw8+ ","+dw9)
#     print(dw1)
#     print(dw2)
#     print(np.round(dw3,6))
#     print(dw4)
#     print(dw5)
#     print(dw6)
#     print(dw7)
#     print(dw8)
#     print(dw9)
#     print(O18)
#     return backward_dict

In [ ]:
def backward_propagation(x,y,w,forward_dict):
    '''In this function, we will compute the backward propagation '''
    # forward_dict: the outputs of the forward_propagation() function
    # write code to compute the gradients of each weight [w1,w2,w3,...,w9]
    # Hint: you can use dict type to store the required variables 
    # dw1 = # in dw1 compute derivative of L w.r.to w1
    # dw2 = # in dw2 compute derivative of L w.r.to w2
    # dw3 = # in dw3 compute derivative of L w.r.to w3
    # dw4 = # in dw4 compute derivative of L w.r.to w4
    # dw5 = # in dw5 compute derivative of L w.r.to w5
    # dw6 = # in dw6 compute derivative of L w.r.to w6
    # dw7 = # in dw7 compute derivative of L w.r.to w7
    # dw8 = # in dw8 compute derivative of L w.r.to w8
    # dw9 = # in dw9 compute derivative of L w.r.to w9
    dw9 = forward_dict['dy_pred'] * 1 * forward_dict['sigmoid']
    dO3O4 = forward_dict['sigmoid'] * (1- forward_dict['sigmoid'])
    dLO4 = forward_dict['dy_pred'] * 1 * w[8] * dO3O4
    dw8 = dLO4 * 1
    dLO6 = forward_dict['dy_pred'] * 1 * (1 - (forward_dict['tanh']**2))
    dw7 = 1 * dLO6
    dLO12 = forward_dict['exp'] * 1 * dLO6
    dw6 = 1 * dLO12
    O17 = math.sin(w[2] * x[2])
    dLO16 = 1 * O17 * 1 * dLO4
    dw5 = x[4] * dLO16
    dLO15 = 1 * O17 * 1 * dLO4
    dw4 = x[3] * dLO15
    O18 = (x[3] * w[3]) + (x[4] * w[4])
    dLO17 = O18 * 1 * dLO4    
    dLO14 = math.cos(x[2] * w[2]) * dLO17
    dw3 = x[2] * dLO14
    #dLO8 = (1 * (-0.140792) * 1 * dLO12) + (1 * (-0.140792) * 1 * dLO12)
    O9 = (w[0] * x[0]) + (w[1] * x[1])
    O10 = (w[0] * x[0]) + (w[1] * x[1])
    dLO8 = (1 * O9 * 1 * dLO12) + (1 * O10 * 1 * dLO12)
    dw2 = x[1] * dLO8
    dLO7 = dLO8
    dw1 = x[0] * dLO7
    backward_dict={}
    #store the variables dw1,dw2 etc. in a dict as backward_dict['dw1']= dw1,backward_dict['dw2']= dw2...
    backward_dict['dw1']= dw1
    backward_dict['dw2']= dw2
    backward_dict['dw3']= dw3
    backward_dict['dw4']= dw4
    backward_dict['dw5']= dw5
    backward_dict['dw6']= dw6
    backward_dict['dw7']= dw7
    backward_dict['dw8']= dw8
    backward_dict['dw9']= dw9
    #print(dw1 + ","+ dw2+ ","+dw3+ ","+dw4+ ","+dw5+ ","+dw6+ ","+dw7+ ","+dw8+ ","+dw9)
    # print(dw1)
    # print(dw2)
    # print(dw3)
    # print(dw4)
    # print(dw5)
    # print(dw6)
    # print(dw7)
    # print(dw8)
    # print(dw9)
    # print(O18)
    return backward_dict

In [ ]:
def grader_backprop(data):
    dw1=(np.round(data['dw1'],6)==-0.229733)
    dw2=(np.round(data['dw2'],6)==-0.021408)
    dw3=(np.round(data['dw3'],6)==-0.005625)
    dw4=(np.round(data['dw4'],6)==-0.004658)
    dw5=(np.round(data['dw5'],6)==-0.001008)
    dw6=(np.round(data['dw6'],6)==-0.633475)
    dw7=(np.round(data['dw7'],6)==-0.561942)
    dw8=(np.round(data['dw8'],6)==-0.048063)
    dw9=(np.round(data['dw9'],6)==-1.018104)
    assert(dw1 and dw2 and dw3 and dw4 and dw5 and dw6 and dw7 and dw8 and dw9)
    return True 
w=np.ones(9)*0.1
forward_dict=forward_propagation(X[0],y[0],w)
backward_dict=backward_propagation(X[0],y[0],w,forward_dict)
grader_backprop(backward_dict)

# <font color='red'>Task 1.3 </font>

 ## <font color='blue'>Gradient clipping</font>

<b> Check this  <a href='https://towardsdatascience.com/how-to-debug-a-neural-network-with-gradient-checking-41deec0357a9'>blog link</a> for more details on Gradient clipping

 we know that the derivative of any function is
 
 $$\lim_{\epsilon\to0}\frac{f(x+\epsilon)-f(x-\epsilon)}{2\epsilon}$$


*  The definition above can be used as a numerical approximation of the derivative. Taking an epsilon small enough, the calculated approximation will have an error in the range of epsilon squared. 

*  In other words, if epsilon is 0.001, the approximation will be off by 0.00001.

Therefore, we can use this to approximate the gradient, and in turn make sure that backpropagation is implemented properly. This forms the basis of <b>gradient checking!</b>

## <Font color='blue'>Gradient checking example</font>

<font >
lets understand the concept with a simple example:
$f(w1,w2,x1,x2)=w_{1}^{2} . x_{1} + w_{2} . x_{2}$ 

from the above function , lets assume $w_{1}=1$, $w_{2}=2$, $x_{1}=3$, $x_{2}=4$ the gradient of $f$ w.r.t $w_{1}$ is

\begin{array} {lcl}
\frac{df}{dw_{1}} = dw_{1} &=&2.w_{1}.x_{1} \\& = &2.1.3\\& = &6
\end{array}


let calculate the aproximate gradient of $w_{1}$ as mentinoned in the above formula and considering $\epsilon=0.0001$

\begin{array} {lcl}
dw_1^{approx} & = & \frac{f(w1+\epsilon,w2,x1,x2)-f(w1-\epsilon,w2,x1,x2)}{2\epsilon} \\ & = & \frac{((1+0.0001)^{2} . 3 + 2 . 4) - ((1-0.0001)^{2} . 3 + 2 . 4)}{2\epsilon} \\ & = & \frac{(1.00020001 . 3 + 2 . 4) - (0.99980001. 3 + 2 . 4)}{2*0.0001} \\ & = & \frac{(11.00060003) - (10.99940003)}{0.0002}\\ & = & 5.99999999999
\end{array}

Then, we apply the following formula for gradient check: <i>gradient_check</i> = 
$\frac{\left\Vert\left (dW-dW^{approx}\rm\right) \right\Vert_2}{\left\Vert\left (dW\rm\right) \right\Vert_2+\left\Vert\left (dW^{approx}\rm\right) \right\Vert_2}$

The equation above is basically the Euclidean distance normalized by the sum of the norm of the vectors. We use normalization in case that one of the vectors is very small.
As a value for epsilon, we usually opt for 1e-7. Therefore, if gradient check return a value less than 1e-7, then it means that backpropagation was implemented correctly. Otherwise, there is potentially a mistake in your implementation. If the value exceeds 1e-3, then you are sure that the code is not correct.

in our example: <i>gradient_check</i> $ = \frac{(6 - 5.999999999994898)}{(6 + 5.999999999994898)} = 4.2514140356330737e^{-13}$

you can mathamatically derive the same thing like this

\begin{array} {lcl}
dw_1^{approx} & = & \frac{f(w1+\epsilon,w2,x1,x2)-f(w1-\epsilon,w2,x1,x2)}{2\epsilon} \\ & = & \frac{((w_{1}+\epsilon)^{2} . x_{1} + w_{2} . x_{2}) - ((w_{1}-\epsilon)^{2} . x_{1} + w_{2} . x_{2})}{2\epsilon} \\ & = & \frac{4. \epsilon.w_{1}. x_{1}}{2\epsilon} \\ & = &  2.w_{1}.x_{1}
\end{array}


## <font color='red'> Implement Gradient checking </font> <br>
 (Write your code in <font color='blue'> def gradient_checking()</font>)

**Algorithm** 



<pre>
<font color='darkblue'>
W = initilize_randomly
def gradient_checking(data_point, W):<font color='grey'>
    # compute the L value using forward_propagation()
    # compute the gradients of W using backword_propagation()</font>
    approx_gradients = []
    for each wi weight value in W:<font color='grey'>
        # add a small value to weight wi, and then find the values of L with the updated weights
        # subtract a small value to weight wi, and then find the values of L with the updated weights
        # compute the approximation gradients of weight wi</font>
        approx_gradients.append(approximation gradients of weight wi)<font color='grey'>
    # compare the gradient of weights W from backword_propagation() with the aproximation gradients of weights with <br>  gradient_check formula</font>
    return gradient_check</font>
<b>NOTE: you can do sanity check by checking all the return values of gradient_checking(),<br> they have to be zero. if not you have bug in your code
</pre></b>

In [ ]:
def gradient_checking(x,y,w,eps):
    # compute the dict value using forward_propagation()
    # compute the actual gradients of W using backword_propagation()
    forward_dict=forward_propagation(x,y,w)
    backward_dict=backward_propagation(x,y,w,forward_dict)
    
    #we are storing the original gradients for the given datapoints in a list
    
    original_gradients_list=list(backward_dict.values())
    # make sure that the order is correct i.e. first element in the list corresponds to  dw1 ,second element is dw2 etc.
    # you can use reverse function if the values are in reverse order
    
    #now we have to write code for approx gradients, here you have to make sure that you update only one weight at a time
    approx_gradients_list=[]
    for i in range(len(w)):
        # add a small value to weight wi, and then find the values of loss_plus with the updated weights
        w_plus=w.copy()
        w_plus[i] = w_plus[i]+eps
        forward_dict = forward_propagation(x,y,w_plus)
        loss_plus   = forward_dict['loss']
        
        # subtract a small value to weight wi, and then find the values of loss_minus with the updated weights
        # please write your code below to calcualte loss_minus
        w_minus=w.copy()
        w_minus[i] = w_minus[i]-eps
        forward_dict = forward_propagation(x,y,w_minus)
        loss_minus   = forward_dict['loss']
        
        # now we will calualte approximate gradient for a particular weight
        approx_gradient = (loss_plus-loss_minus)/(2*eps)
        approx_gradients_list.append(approx_gradient)
    
    #performing gradient check operation
    original_gradients_list=np.array(original_gradients_list)
    approx_gradients_list=np.array(approx_gradients_list)
    gradient_check_value =(original_gradients_list-approx_gradients_list)/(original_gradients_list+approx_gradients_list)
    
    return gradient_check_value
    

In [ ]:
def grader_grad_check(value):
    print(value)
    assert(np.all(value < 10**-4))
    return True 

w=[ 0.00271756,  0.01260512,  0.00167639, -0.00207756,  0.00720768,
   0.00114524,  0.00684168,  0.02242521,  0.01296444]

eps=10**-7
value= gradient_checking(X[0],y[0],w,eps)
grader_grad_check(value)

# <font color='red'> Task 2 : Optimizers </font>

* As a part of this task, you will be implementing 2  optimizers(methods to update weight)
* Use the same computational graph that was mentioned above to do this task
* The weights have been initialized from normal distribution with mean=0 and std=0.01. The initialization of weights is very important otherwiswe you can face vanishing gradient and exploding gradients problem.

**Check below video for reference purpose**

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('gYpoJMlgyXA',width="1000",height="500")

<font color='blue'><b>Algorithm</b>

<pre>
    for each epoch(1-10):
        for each data point in your data:
            using the functions forward_propagation() and backword_propagation() compute the gradients of weights
            update the weigts with help of gradients  
</pre>

## <font color='red'> Implement below tasks</b>


*  <b>Task 2.1</b>: you will be implementing the above algorithm with <b>Vanilla update</b> of weights<br><br>
*  <b>Task 2.2</b>: you will be implementing the above algorithm with <b>Momentum update</b> of weights<br><br>



**Note : If you get any assertion error while running grader functions, please print the variables in grader functions and check which variable is returning False .Recheck your logic for that variable .**

###<font color='blue'>2.1 Algorithm with Vanilla update of weights</font>

In [ ]:
def vanilla_update(w_vanilla,X,Y,learning_rate,epochs):
    mean_loss=[]
    for e in range(epochs):
        loss_per_datapoint=0
        for i in range(len(X)):
            #calculate forward propogation
            forward_dict = forward_propagation(X[i], y[i], w_vanilla)  
            #adding loss for each datapoint
            loss_per_datapoint+=forward_dict['loss']    
            #calculating gradient dict using backward propogation
            gradients = backward_propagation(X[i],y[i],w_vanilla,forward_dict)   
            #getting the values of gradients from dictionary
            dw = np.array(list(gradients.values()))                                    
            
            #update w according to vanilla update => w_new = w_old - learning_rate*dw
            #you have to write your code here to update weights according to vanilla optimizer
            for j in range(len(w_vanilla)):
              w_vanilla[j] = w_vanilla[j] - (learning_rate * dw[j])            
        # we are appending the average loss for all the datapoints
        mean_loss.append(loss_per_datapoint/len(X))
    return np.array(mean_loss)

In [ ]:
def grader_grad_check(array):
    assert(np.round(array[0],3)==0.678 and np.round(array[-1],3)==0.0310)
    return True 

w_vanilla=np.array([ 0.00244274 ,0.00973789,-0.00504212, 0.00146322,-0.00952112, 0.03329146,
 -0.00042503  ,0.00149911 ,0.0136601 ])
learning_rate=0.001
epochs=10
vanilla_loss_array= vanilla_update(w_vanilla,X,y,learning_rate,epochs)
print(grader_grad_check(vanilla_loss_array))
plt.plot(np.arange(epochs),vanilla_loss_array)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

###<font color='blue'>2.2 Algorithm with Momentum update of weights</font>

<img src='https://i.imgur.com/gyPSXhS.png'>

Here Gamma referes to the momentum coefficient, eta is leaning rate and v_t is moving average of our gradients at timestep t

In [ ]:
def momentum_update(w_momentum,X,Y,learning_rate,epochs,v,gamma):
    mean_loss=[]
    for e in range(epochs):
        loss_per_datapoint=0
        for i in range(len(X)):
            #calculate forward propogation
            forward_dict = forward_propagation(X[i], y[i], w_momentum)  
            #adding loss for each datapoint
            loss_per_datapoint+=forward_dict['loss']    
            #calculating gradient dict using backward propogation
            gradients = backward_propagation(X[i],y[i],w_momentum,forward_dict)   
            #getting the values of gradients from dictionary
            dw = np.array(list(gradients.values()))                        

            #you have to write your code here to update weights according to momentum optimizer
            for j in range(len(w_momentum)):
              #change = dw[j] * step_size            
              #w_momentum[j] = w_momentum[j] - change

              # Momentum update
              v[j] = gamma * v[j] - learning_rate * dw[j] # integrate velocity
              w_momentum[j] += v[j] # integrate position

            
        # we are appending the average loss for all the datapoints
        mean_loss.append(loss_per_datapoint/len(X))
    return np.array(mean_loss)

In [ ]:
def grader_grad_check(array):
    assert(np.round(array[0],3)==0.143 and np.round(array[-1],3)==0.005)
    return True 

w_momentum=np.array([ 0.00244274 ,0.00973789,-0.00504212, 0.00146322,-0.00952112, 0.03329146,
 -0.00042503  ,0.00149911 ,0.0136601 ])

learning_rate=0.001
epochs=10
v=np.zeros(9)                         
m=0.9
momentum_loss_array= momentum_update(w_momentum,X,y,learning_rate,epochs,v,m)
print(grader_grad_check(momentum_loss_array))
plt.plot(np.arange(epochs),momentum_loss_array)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

<font color='blue'>Comparision plot between epochs and loss with different optimizers</font>

In [ ]:

plt.plot(vanilla_loss_array, label = 'Vanilla Update')
plt.plot(momentum_loss_array, label = 'Momentum Update')


plt.title('Comparision b/w epochs & loss with different Optimizers')
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.legend()
plt.show()

<font color='blue'><b>You can go through the following blog to understand the implementation of other optimizers well</font>
   <br> [Gradients update blog](https://cs231n.github.io/neural-networks-3/) </br>